In [1]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [2]:
GROQ_KEY="write your key here

In [3]:
import json
import datetime
import os

from groq import Groq
date = datetime.datetime.now()
date
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

In [16]:
%%writefile trans_summarize.py

import datetime
from groq import Groq
GROQ_KEY = "write your key here"
client = Groq(
    api_key=GROQ_KEY,
)
MODEL = 'llama3-70b-8192'

def run_conversation(user_prompt,task):
      sys_prompt=""
      if task=="Summarization":
        sys_prompt = """
          You are an intelligent assistant capable of translating text between different languages and summarizing text.

          **Summarization Task**: If the user asks to summarize a text, provide a concise summary of the provided text.

          **Guidelines**:
          - Always identify the summarization based on the user's prompt.
          - Respond directly with the summary without additional commentary.
          """
      else:
        sys_prompt = """
      You are an intelligent assistant capable of translating text between different languages.

      1. **Translation Task**: If the user asks to translate a text, translate the given text from the source language to the target language specified by the user.

      **Guidelines**:
      - Always identify the translation based on the user's prompt.
      - Respond directly with the translated text without additional commentary.
      """

      messages = [
        {
            "role": "system",
            "content": sys_prompt
        },
    ]
      messages.append({
        "role": "user",
        "content": user_prompt
    })

      response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=4096,
        temperature=0,
    )

      response_message = response.choices[0].message
      return response_message.content

Writing trans_summarize.py


In [5]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [22]:
%%writefile app.py
import streamlit as st
from trans_summarize import run_conversation

def main():

    st.title("Text Processing App")

    task = st.selectbox("Select Task", ["Summarization", "Translation"])

    language = None
    if task == "Translation":
        language = st.selectbox("Select Target Language", ["Spanish", "French", "German", "Chinese","Urdu"])

    user_input = st.text_area("Enter the text you want to process")


    if st.button("Process"):
        if task == "Summarization":
            input_text = f"summarize this: {user_input}"
        elif task == "Translation" and language:
            input_text = f"translate to {language.lower()}: {user_input}"
        else:
            st.error("Please select a valid task and provide text.")
            return
        bot_response = run_conversation(input_text,task)

        st.write("**Result:**")
        st.write(bot_response)


if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
import os
from pyngrok import ngrok
ngrok.set_auth_token('write your ngrok auth key')

port = 8500  

public_url = ngrok.connect(port)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
os.system(f'streamlit run app.py --server.port {port}')


Streamlit app is live at: NgrokTunnel: "https://0ca8-35-197-106-1.ngrok-free.app" -> "http://localhost:8500"
